In [1]:
"""
Define our CalcOpr function and the Match class that we use to interface with it
"""

from collections import namedtuple
import numpy.matlib as mat
import numpy as np
from numpy.linalg import inv

Match = namedtuple("Match", ["teams", "score"])

def CalcOpr(matches):
    """
    Given a list of matches, to least squares OPR calculation
    """
    num_matches = len(matches) # rows
    teams = list(set().union(*[match.teams for match in matches]))
    num_teams = len(teams)
    
    alliances = mat.zeros((num_matches, num_teams))
    scores = mat.zeros((num_matches, 1))
    
    for idx, match in enumerate(matches):
        scores[idx, 0] = match.score
        for team in match.teams:
            alliances[idx, teams.index(team)] = 1
        
    least_squares_approx = np.dot(
        inv(np.dot(np.transpose(alliances), alliances)),
        np.dot(np.transpose(alliances), scores))
    
    oprs = {}
    for idx, team in enumerate(teams):
        oprs[team] = least_squares_approx[idx, 0] 
    return oprs

print(CalcOpr([
            Match(teams=['A', 'B'], score=10),
            Match(teams=['A', 'C'], score=13),
            Match(teams=['B', 'C'], score=7),
            Match(teams=['A', 'D'], score=15),
            Match(teams=['B', 'D'], score=10),
        ]))

{'C': 5.0, 'D': 7.5, 'A': 7.75, 'B': 2.25}


In [2]:
"""
Connect to the blue alliance api.  Also install a cache so that every request
we make is cached to a file for a day.  This doesn't respect the cache headers
and last-changed headers from tba's api, but since we're using old data we can
be pretty sure that the data we're using is fresh.
"""

import requests
import requests_cache

# Cache all api requests in a file called tba_cache.sqlite for a day
# (monkeypatch the requests module that tbapy uses)
requests_cache.install_cache(cache_name="tba_cache", expire_after=60 * 60 * 24)

import tbapy
from pprint import pprint

api_key = "6qOZ9uAEsb4CDrOBNG6ZnIdi9cWBaZ6DHnCSato97Qfo7bBeUwT9NfFt4Gi5sHFN"

tba = tbapy.TBA(api_key)
pprint(tba.status())

{'android': {'latest_app_version': 4020399, 'min_app_version': 4000299},
 'contbuild_enabled': True,
 'current_season': 2018,
 'down_events': [],
 'ios': {'latest_app_version': -1, 'min_app_version': -1},
 'is_datafeed_down': False,
 'json': {'android': {'latest_app_version': 4020399,
                      'min_app_version': 4000299},
          'contbuild_enabled': True,
          'current_season': 2018,
          'down_events': [],
          'ios': {'latest_app_version': -1, 'min_app_version': -1},
          'is_datafeed_down': False,
          'max_season': 2018,
          'web': {'commit_time': '2018-02-09 14:37:30 -0500',
                  'current_commit': '3dd2c80b8ec53047bdef65b60929b6293b9e98a6',
                  'deploy_time': 'Fri Feb  9 19:51:54 UTC 2018',
                  'travis_job': '339614111'}},
 'max_season': 2018,
 'web': {'commit_time': '2018-02-09 14:37:30 -0500',
         'current_commit': '3dd2c80b8ec53047bdef65b60929b6293b9e98a6',
         'deploy_time': 'Fri 

In [3]:
"""
Given the matches from an event, let's calculate the OPR for each component.
Current components are raw score, rotor rp, and fuel rp.  
"""


def CalcScoreOpr(matches):
    """
    Takes in Matches (like what tba.event_matches returns) and outputs a dictionary
    mapping team-name to score opr
    """
    match_outcomes = []
    for match in matches:
        match_outcomes.append(
            Match(teams=match['alliances']['blue']['team_keys'],
                  score=match['alliances']['blue']['score'])
        )
        match_outcomes.append(
            Match(teams=match['alliances']['red']['team_keys'],
                  score=match['alliances']['red']['score'])
        )
    return CalcOpr(match_outcomes)


def CalcRotorRpOpr(matches):
    """
    Calculate each team's opr with respect to getting a robot bonus.  Rotor bonus is
    pretty nonlinear so part of the experiment here is seeing how well we can predict
    an alliance's liklihood of getting the rotor RP
    """
    match_outcomes = []
    for match in matches:
        if match['comp_level'] == 'qm':
            match_outcomes.append(
                Match(teams=match['alliances']['blue']['team_keys'],
                      score=match['score_breakdown']['blue']['rotorRankingPointAchieved'])
            )
            match_outcomes.append(
                Match(teams=match['alliances']['red']['team_keys'],
                      score=match['score_breakdown']['red']['rotorRankingPointAchieved'])
            )
    return CalcOpr(match_outcomes)
    
    
def CalcFuelRpOpr(matches):
    """
    Calculate each team's opr with respect to getting a fuel bonus.  Fuel bonus
    is pretty nonlinear so part of the experiment here is seeing how well we can predict
    an alliance's likliood of getting the fuel RP
    """
    match_outcomes = []
    for match in matches:
        if match['comp_level'] == 'qm':
            match_outcomes.append(
                Match(teams=match['alliances']['blue']['team_keys'],
                      score=match['score_breakdown']['blue']['kPaRankingPointAchieved'])
            )
            match_outcomes.append(
                Match(teams=match['alliances']['red']['team_keys'],
                      score=match['score_breakdown']['red']['kPaRankingPointAchieved'])
            )
    return CalcOpr(match_outcomes)

In [4]:
from collections import namedtuple

EventSummary = namedtuple("EventSummary", ["team_score_opr", "team_rotor_rp_opr", "team_fuel_rp_opr"])

_event_stats = {}

def get_event_stats(event):
    if event not in _event_stats:
        event_matches = tba.event_matches(event)

        _event_stats[event] = EventSummary(
            team_score_opr = CalcScoreOpr(event_matches),
            team_rotor_rp_opr = CalcRotorRpOpr(event_matches),
            team_fuel_rp_opr = CalcFuelRpOpr(event_matches),
        )
    
    return _event_stats[event]
    
pprint(dict(get_event_stats("2017roe")._asdict()))

{'team_fuel_rp_opr': {'frc1002': 0.0013067540141778449,
                      'frc1011': -0.006691819430700594,
                      'frc115': -0.00027964533450865149,
                      'frc1339': 0.027811071747965703,
                      'frc1414': -0.028274925604767444,
                      'frc1477': 0.030186389459701268,
                      'frc1482': 0.0038428145423401103,
                      'frc1574': 0.81224332275922417,
                      'frc175': 0.0086009350495095428,
                      'frc2183': 0.0017301384649544664,
                      'frc2403': 0.097203821426750756,
                      'frc2468': -0.00038628735476987398,
                      'frc2478': 0.045443848842528743,
                      'frc2485': -0.091623608856357577,
                      'frc2642': 0.019303911285171359,
                      'frc2655': 0.025380401944398667,
                      'frc2881': 0.057278301432307215,
                      'frc2905': -0.020735668480764094,

In [5]:
TeamSummary = namedtuple("TeamSummary", ["last_event", "score_opr", "rotor_rp_opr", "fuel_rp_opr"])

_team_stats = {}

def get_team_last_event(team, year="2017"):
    events = tba.team_events(team, year)
    events = [
        event
        for event
        in events
        if event['event_type_string'] in ['Regional', 'District Championship', 'District']
    ]
    
    events = sorted(events, key=lambda x: x['end_date'])
    return events[-1]['event_code']

def get_team_stats(team, year="2017"):
    """
    Get the component opr for the given team based on their most recent regional,
    district champ, or district event
    """
    if team not in _team_stats:
        team_last_event = get_team_last_event(team, year)
        eventSummary = get_event_stats(year + team_last_event)
        
        teamSummary = TeamSummary(
            last_event = team_last_event,
            score_opr = eventSummary.team_score_opr[team],
            rotor_rp_opr = eventSummary.team_rotor_rp_opr[team],
            fuel_rp_opr = eventSummary.team_fuel_rp_opr[team],
        )
        
        _team_stats[team] = teamSummary
    
    return _team_stats[team]

get_team_stats('frc973')

TeamSummary(last_event='cada', score_opr=120.68964289866561, rotor_rp_opr=0.0, fuel_rp_opr=0.44310225380396301)

In [6]:
"""
Simulate one match given the red and blue alliances and creating a MatchOutcome object
"""


import random

MatchOutcome = namedtuple("MatchOutcome", [
        "red_alliance", "blue_alliance",
        "red_score", "blue_score",
        "red_match_rp", "blue_match_rp",
        "red_fuel_rp", "blue_fuel_rp", 
        "red_rotor_rp", "blue_rotor_rp",
        "red_total_rp", "blue_total_rp"
    ])

def simulate_match(red_alliance, blue_alliance):
    red_reports = [get_team_stats(team) for team in red_alliance]
    blue_reports = [get_team_stats(team) for team in blue_alliance]
    
    red_score = sum([
            report.score_opr * random.gauss(1.0, 0.3)
            for report
            in red_reports
    ]) + random.gauss(0.0, 10.0)
    
    blue_score = sum([
            report.score_opr * random.gauss(1.0, 0.3)
            for report
            in blue_reports
    ]) + random.gauss(0.0, 10.0)
    
    if red_score > blue_score:
        red_match_rp = 2
        blue_match_rp = 0
    elif blue_score > red_score:
        blue_match_rp = 2
        red_match_rp = 0
    else:
        blue_match_rp = 1
        red_match_rp = 1
        
    red_rotor_rp_chance = sum([
        report.rotor_rp_opr for report in red_reports
    ])
        
    blue_rotor_rp_chance = sum([
        report.rotor_rp_opr for report in blue_reports
    ])
    
    red_rotor_rp = int(red_rotor_rp_chance > random.random())
    blue_rotor_rp = int(blue_rotor_rp_chance > random.random())
        
    red_fuel_rp_chance = sum([
        report.fuel_rp_opr for report in red_reports
    ])
        
    blue_fuel_rp_chance = sum([
        report.fuel_rp_opr for report in blue_reports
    ])
    
    red_fuel_rp = int(red_fuel_rp_chance > random.random())
    blue_fuel_rp = int(blue_fuel_rp_chance > random.random())
    
    return MatchOutcome(
        red_alliance = red_alliance, blue_alliance = blue_alliance,
        red_score = red_score, blue_score = blue_score,
        red_match_rp = red_match_rp, blue_match_rp = blue_match_rp,
        red_fuel_rp = red_fuel_rp, blue_fuel_rp = blue_fuel_rp,
        red_rotor_rp = red_rotor_rp, blue_rotor_rp = blue_rotor_rp,
        red_total_rp = red_match_rp + red_fuel_rp + red_rotor_rp,
        blue_total_rp = blue_match_rp + blue_fuel_rp + blue_rotor_rp,
    )

for _ in range(3):
    pprint(simulate_match(["frc973", "frc1011", "frc492"], ["frc254", "frc1678", "frc294"])._asdict())

OrderedDict([('red_alliance', ['frc973', 'frc1011', 'frc492']),
             ('blue_alliance', ['frc254', 'frc1678', 'frc294']),
             ('red_score', 327.77351158037442),
             ('blue_score', 357.81508687313874),
             ('red_match_rp', 0),
             ('blue_match_rp', 2),
             ('red_fuel_rp', 0),
             ('blue_fuel_rp', 1),
             ('red_rotor_rp', 1),
             ('blue_rotor_rp', 0),
             ('red_total_rp', 1),
             ('blue_total_rp', 3)])
OrderedDict([('red_alliance', ['frc973', 'frc1011', 'frc492']),
             ('blue_alliance', ['frc254', 'frc1678', 'frc294']),
             ('red_score', 408.00976802947906),
             ('blue_score', 433.90100072579713),
             ('red_match_rp', 0),
             ('blue_match_rp', 2),
             ('red_fuel_rp', 0),
             ('blue_fuel_rp', 1),
             ('red_rotor_rp', 1),
             ('blue_rotor_rp', 0),
             ('red_total_rp', 1),
             ('blue_total_rp', 3)]

In [7]:
def get_schedule(event):
    """
    Given an event key return the schedule (a list of (red_alliance, blue_alliance) tuples)
    """
    schedule = []
    event_matches = tba.event_matches(event)
    for match in event_matches:
        schedule.append((match['alliances']['red']['team_keys'],
                         match['alliances']['blue']['team_keys']))
    
    return schedule

def simulate_schedule(schedule):
    """
    Given a schedule, simulate the outcomes of all matches
    """
    outcomes = []
    for match in schedule:
        outcomes.append(simulate_match(match[0], match[1]))
    return outcomes

def get_rankings(match_outcomes):
    "Based on the outcomes of each match, determine the rankings"
    teams = set().union(*[match.red_alliance for match in match_outcomes])
    
    team_rps = []
    for team in teams:
        num_rps = 0
        for outcome in match_outcomes:
            if team in outcome.red_alliance:
                num_rps += outcome.red_total_rp
            if team in outcome.blue_alliance:
                num_rps += outcome.blue_total_rp
        team_rps.append((team, num_rps))
    team_rps.sort(key=lambda x: -x[1])
    
    rankings = {}
    for idx, (team, rps) in enumerate(team_rps):
        rankings[team] = idx + 1
        
    return rankings
        
def simulate_event(event):
    return get_rankings(simulate_schedule(get_schedule(event)))

pprint(simulate_event("2017roe"))

{'frc1002': 21,
 'frc1011': 6,
 'frc115': 9,
 'frc1339': 43,
 'frc1414': 45,
 'frc1477': 36,
 'frc1482': 59,
 'frc1574': 2,
 'frc175': 49,
 'frc2183': 57,
 'frc2403': 52,
 'frc2468': 15,
 'frc2478': 12,
 'frc2485': 53,
 'frc2642': 18,
 'frc2655': 41,
 'frc2881': 56,
 'frc2905': 11,
 'frc2928': 3,
 'frc3140': 46,
 'frc3158': 44,
 'frc3229': 27,
 'frc3316': 23,
 'frc3402': 47,
 'frc365': 7,
 'frc3653': 32,
 'frc3824': 10,
 'frc3826': 28,
 'frc3834': 55,
 'frc3991': 34,
 'frc4060': 58,
 'frc418': 4,
 'frc4191': 64,
 'frc4219': 33,
 'frc4265': 35,
 'frc4276': 38,
 'frc435': 19,
 'frc4371': 60,
 'frc441': 40,
 'frc4561': 5,
 'frc4590': 54,
 'frc4592': 24,
 'frc4723': 48,
 'frc488': 25,
 'frc5026': 31,
 'frc5472': 22,
 'frc5499': 14,
 'frc5515': 39,
 'frc5614': 20,
 'frc5803': 29,
 'frc5816': 50,
 'frc585': 51,
 'frc5970': 26,
 'frc6144': 62,
 'frc624': 8,
 'frc6304': 66,
 'frc6325': 63,
 'frc6361': 65,
 'frc6388': 42,
 'frc6409': 61,
 'frc6508': 13,
 'frc6560': 37,
 'frc6705': 16,
 'frc8': 

In [11]:
AggregateRanking = namedtuple("AggregateRanking", ["p0", "p100", "p50", "p75", "p25"])

def monte_carlo_event(event, iterations=1000):
    team_ranks = {}
    
    for _ in range(iterations):
        rankings = simulate_event(event)
        for team, rank in rankings.items():
            if team not in team_ranks:
                team_ranks[team] = []
            team_ranks[team].append(rank)
    
    aggregate_rankings = {}
    for team, rankings in team_ranks.items():
        aggregate_rankings[team] = AggregateRanking(
            p0=np.percentile(rankings, 0),
            p100=np.percentile(rankings, 100),
            p50=np.percentile(rankings, 50),
            p75=np.percentile(rankings, 75),
            p25=np.percentile(rankings, 25),
        )
    
    return aggregate_rankings

def display_monte_carlo_result(monte_carlo_result):
    monte_carlo_result = list(monte_carlo_result.items())
    monte_carlo_result.sort(key=lambda x: (x[1].p50, x[1].p75))
    
    for team, agr_rank in monte_carlo_result:
        print("Rank {0:2.0f}: {1:10s} (25p={2:2.0f}, 75p={3:2.0f})".format(agr_rank.p50, team, agr_rank.p25, agr_rank.p75))
    
    pprint(monte_carlo_result)

display_monte_carlo_result(monte_carlo_event("2017roe", iterations=1000))

Rank  2: frc1574    (25p= 1, 75p= 3)
Rank  2: frc973     (25p= 1, 75p= 4)
Rank  4: frc365     (25p= 2, 75p= 6)
Rank  6: frc2928    (25p= 4, 75p=10)
Rank  6: frc1011    (25p= 4, 75p=10)
Rank  7: frc4561    (25p= 5, 75p=12)
Rank  9: frc115     (25p= 6, 75p=14)
Rank 10: frc418     (25p= 7, 75p=14)
Rank 10: frc2468    (25p= 6, 75p=15)
Rank 11: frc6705    (25p= 7, 75p=17)
Rank 12: frc624     (25p= 8, 75p=18)
Rank 12: frc1002    (25p= 8, 75p=18)
Rank 13: frc4265    (25p= 8, 75p=18)
Rank 13: frc8       (25p= 8, 75p=18)
Rank 15: frc4592    (25p=10, 75p=20)
Rank 16: frc3824    (25p=11, 75p=22)
Rank 18: frc5499    (25p=14, 75p=25)
Rank 19: frc1477    (25p=13, 75p=26)
Rank 22: frc2478    (25p=15, 75p=30)
Rank 24: frc6508    (25p=18, 75p=31)
Rank 24: frc1414    (25p=18, 75p=31)
Rank 25: frc3229    (25p=19, 75p=32)
Rank 26: frc488     (25p=18, 75p=34)
Rank 26: frc3653    (25p=20, 75p=33)
Rank 26: frc5026    (25p=19, 75p=34)
Rank 26: frc2642    (25p=19, 75p=34)
Rank 28: frc2905    (25p=21, 75p=35)
R